Insert picture of disney

In [2]:
%run ./Includes/Classroom-Setup

In [3]:
spark.sql("DROP TABLE IF EXISTS app_info_csv")

schemaDDL = "APP STRING, CATEGORY STRING, RATING FLOAT, REVIEWS INT, SIZE STRING, INSTALLS STRING, TYPE STRING, PRICE STRING, CONTENT_RATING STRING,GENRES STRING,lAST_UPDATED STRING,CURRENT_VERSION STRING,ANDROID_VERSION STRING"

# File location and type
sourcePath = "/FileStore/tables/app_info.csv"
file_type = "csv"

tablePath = f"{userhome}/app_info_csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","


(spark.read
  .format("csv")
  .schema(schemaDDL)
  .load(sourcePath)
  .write
  .option("path", tablePath)
  .mode("overwrite")
  .saveAsTable("app_info_csv"))

appinfodf = spark.table("app_info_csv")

In [4]:
#Answer 1
#Cleaning



appinfodf.printSchema()

root
-- APP: string (nullable = true)
-- CATEGORY: string (nullable = true)
-- RATING: float (nullable = true)
-- REVIEWS: integer (nullable = true)
-- SIZE: string (nullable = true)
-- INSTALLS: string (nullable = true)
-- TYPE: string (nullable = true)
-- PRICE: string (nullable = true)
-- CONTENT_RATING: string (nullable = true)
-- GENRES: string (nullable = true)
-- lAST_UPDATED: string (nullable = true)
-- CURRENT_VERSION: string (nullable = true)
-- ANDROID_VERSION: string (nullable = true)

In [5]:
appinfodf.count()

Out[22]: 10842

In [6]:
#Removed duplicates
dfdistinct = appinfodf.distinct()
dfdistinct.count()


Out[23]: 10356

Data collection is poor and needs to be improved
For example, the Genre contains comma separated values: Art & Design;Creativity 
Ratings contain NAN values

Version column contains Varies with device value
It is advisable to assign id to the rows so that we can mainpulate each column and join the columns back based on keys.


For EDA, We need to convert many columns into Categorical data. For example,

Content Rating, Installs, Android Version. 
Also, price datatype needs to be fixed.

In [8]:
# Ratings is a main column, equivalient to Quality of a product.
# NAN rows need to be fixed. 


In [9]:
from pyspark.sql.functions import *

## Preview 20 lines of data

In [11]:

display(dfdistinct.limit(100))

APP,CATEGORY,RATING,REVIEWS,SIZE,INSTALLS,TYPE,PRICE,CONTENT_RATING,GENRES,lAST_UPDATED,CURRENT_VERSION,ANDROID_VERSION
Girls Hairstyles,BEAUTY,4.1,3595,Varies with device,"500,000+",Free,0,Everyone,Beauty,"May 24, 2018",Varies with device,4.0 and up
Facial Wrinkle Reduction,BEAUTY,4.6,184,21M,"10,000+",Free,0,Everyone,Beauty,"July 2, 2018",1.5,4.1 and up
OfficeSuite Pro + PDF (Trial),BUSINESS,4.2,88073,Varies with device,"10,000,000+",Free,0,Everyone,Business,"July 9, 2018",Varies with device,Varies with device
ClickMeeting Webinars,BUSINESS,3.8,5211,26M,"1,000,000+",Free,0,Everyone,Business,"July 30, 2018",3.3.0,4.0 and up
"Moco - Chat, Meet People",DATING,4.2,313724,Varies with device,"10,000,000+",Free,0,Mature 17+,Dating,"August 4, 2018",2.6.141,4.1 and up
ABC Preschool Free,EDUCATION,3.8,27572,25M,"5,000,000+",Free,0,Everyone,Education;Education,"October 25, 2017",3.0,2.3 and up
Colorfy: Coloring Book for Adults - Free,ENTERTAINMENT,4.5,787177,Varies with device,"10,000,000+",Free,0,Everyone,Entertainment,"June 20, 2018",Varies with device,Varies with device
The Bump Pregnancy Tracker,HEALTH_AND_FITNESS,4.6,20301,22M,"1,000,000+",Free,0,Everyone,Health & Fitness,"May 16, 2018",3.22,4.3 and up
DIY On A Budget,HOUSE_AND_HOME,4.7,114,8.3M,"10,000+",Free,0,Teen,House & Home,"February 18, 2018",2.1,4.1 and up
Tattoodo - Find your next tattoo,LIFESTYLE,4.5,7457,13M,"1,000,000+",Free,0,Teen,Lifestyle,"August 3, 2018",2.4.12-r244,4.1 and up


In [12]:
%sql
-- ANSWER

SELECT * 
FROM app_info_csv
LIMIT 20

APP,CATEGORY,RATING,REVIEWS,SIZE,INSTALLS,TYPE,PRICE,CONTENT_RATING,GENRES,lAST_UPDATED,CURRENT_VERSION,ANDROID_VERSION
null,null,null,null,null,null,null,null,null,null,null,null,null
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up


## Replace Ratings NaN with 0

In [15]:
%sql
create view v1 as
select *  from  app_info_csv WHERE rating not in ('NaN')

In [16]:
%sql
select count(*) from v1

count(1)
9364


In [17]:
#Answer 3
# Ratings of top 10 applications which have the most installs

File "<command-2952099777152212>" , line 4 
 select app, rating, installs from v1 
 ^
 SyntaxError : invalid syntax

In [18]:
%sql
select app, rating, installs from v1 order by installs desc limit 10

app,rating,installs
Viber Messenger,4.3,"500,000,000+"
imo free video calls and chat,4.3,"500,000,000+"
imo free video calls and chat,4.3,"500,000,000+"
Google Duo - High Quality Video Calls,4.6,"500,000,000+"
UC Browser - Fast Download Private & Secure,4.5,"500,000,000+"
imo free video calls and chat,4.3,"500,000,000+"
Viber Messenger,4.3,"500,000,000+"
LINE: Free Calls & Messages,4.2,"500,000,000+"
UC Browser - Fast Download Private & Secure,4.5,"500,000,000+"
Viber Messenger,4.3,"500,000,000+"


In [19]:
%sql

/* Query the created temp table in a SQL cell */

select * from `app_info_csv`;

App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


In [20]:
app_info_df = dfdistinct.toPandas()


app_info_df

,APP,CATEGORY,RATING,REVIEWS,SIZE,INSTALLS,TYPE,PRICE,CONTENT_RATING,GENRES,lAST_UPDATED,CURRENT_VERSION,ANDROID_VERSION
0,Girls Hairstyles,BEAUTY,4.1,3595.0,Varies with device,"500,000+",Free,0,Everyone,Beauty,"May 24, 2018",Varies with device,4.0 and up
1,Facial Wrinkle Reduction,BEAUTY,4.6,184.0,21M,"10,000+",Free,0,Everyone,Beauty,"July 2, 2018",1.5,4.1 and up
2,OfficeSuite Pro + PDF (Trial),BUSINESS,4.2,88073.0,Varies with device,"10,000,000+",Free,0,Everyone,Business,"July 9, 2018",Varies with device,Varies with device
3,ClickMeeting Webinars,BUSINESS,3.8,5211.0,26M,"1,000,000+",Free,0,Everyone,Business,"July 30, 2018",3.3.0,4.0 and up
4,"Moco - Chat, Meet People",DATING,4.2,313724.0,Varies with device,"10,000,000+",Free,0,Mature 17+,Dating,"August 4, 2018",2.6.141,4.1 and up
5,ABC Preschool Free,EDUCATION,3.8,27572.0,25M,"5,000,000+",Free,0,Everyone,Education;Education,"October 25, 2017",3.0,2.3 and up
6,Colorfy: Coloring Book for Adults - Free,ENTERTAINMENT,4.5,787177.0,Varies with device,"10,000,000+",Free,0,Everyone,Entertainment,"June 20, 2018",Varies with device,Varies with device
7,The Bump Pregnancy Tracker,HEALTH_AND_FITNESS,4.6,20301.0,22M,"1,000,000+",Free,0,Everyone,Health & Fitness,"May 16, 2018",3.22,4.3 and up
8,DIY On A Budget,HOUSE_AND_HOME,4.7,114.0,8.3M,"10,000+",Free,0,Teen,House & Home,"February 18, 2018",2.1,4.1 and up
9,Tattoodo - Find your next tattoo,LIFESTYLE,4.5,7457.0,13M,"1,000,000+",Free,0,Teen,Lifestyle,"August 3, 2018",2.4.12-r244,4.1 and up


In [21]:
app_groups = app_info_df.groupby('APP')['RATING'].sum().reset_index()
#top_rated_app = app_groups[app_groups.RATING == app_groups.Rating.max() ]


print(app_groups)
print(app_groups['App'].max())
print(top_rated_app['App'].values)
top_rated_app_categories = app_info_df[app_info_df['App'].values == top_rated_app['App'].values]
categories_with_ratings = top_rated_app_categories.groupby('Category')['Rating'].sum()
print(categories_with_ratings)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2952099777152215> in <module> 
 1 app_groups = app_info_df . groupby ( 'APP' ) [ 'RATING' ] . sum ( ) . reset_index ( ) 
 ----> 2 top_rated_app = app_groups [ app_groups . RATING == app_groups . Rating . max ( ) ] 
 3 
 4 
 5 print ( app_groups ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/generic.py in __getattr__ (self, name) 
 5065 if self . _info_axis . _can_hold_identifiers_and_holds_name ( name ) : 
 5066 return self [ name ] 
 -> 5067 return object . __getattribute__ ( self , name ) 
 5068 
 5069 def __setattr__ ( self , name , value ) : 

 AttributeError : 'DataFrame' object has no attribute 'Rating'

In [22]:
app_groups = app_info_df.groupby('APP')['RATING'].sum().reset_index()
#top_rated_app = app_groups[app_groups.RATING == app_groups.Rating.max() ]


print(app_groups)

APP RATING
0 """i DT"" Fútbol. Todos Somos Técnicos." 0.0
1 "Alphabet ""H"" Passcode Lock Screen" 4.5
2 "Eat Fast Prepare ""Without Internet""" 4.6
3 "Official QR Code® Reader ""Q""" 4.4
4 "The FN ""Baby"" pistol explained" 0.0
5 +Download 4 Instagram Twitter 4.5
6 - Free Comics - Comic Apps 3.5
7 .R 4.5
8 /u/app 4.7
9 058.ba 4.4
10 1. FC Köln App 4.6
11 10 Best Foods for You 4.0
12 10 Minutes a Day Times Tables 4.1
13 10 WPM Amateur ham radio CW Morse code trainer 3.5
14 10,000 Quotes DB (Premium) 4.1
15 100 Doors of Revenge 4.1
16 100+ C Programs 4.2
17 100000+ Messages - DP, Status, Jokes & GIF 2018 3.7
18 101 C Programming Problems 4.6
19 104 Looking for a job - looking for a job, loo... 4.4
20 11st 3.8
21 12 Step Meditations & Sober Prayers AA NA AL-ANON 4.7
22 14thStreetVet 0.0
23 17th Edition Cable Sizer 4.4
24 1800 Contacts - Lens Store 4.7
25 1LINE – One Line with One Touch 4.6
26 1st Fed CI Mobile Banking 0.0
27 2 Amateur ham radio CW Morse code practice key... 4.8
28 2-Player Co-op Zombie Shoot 4.2
29 20 Minuten (CH) 3.5
... ... ...
9627 Šmelina .cz inzeráty inzerce 3.9
9628 Аim Training for CS 3.6
9629 Билеты ПДД CD 2019 PRO 0.0
9630 РИА Новости 4.5
9631 РееI Smart Remote MP3 CD Player 3.2
9632 أحداث وحقائق | خبر عاجل في اخبار العالم 4.8
9633 الفاتحون Conquerors 4.5
9634 صور حرف H 4.4
9635 لعبة تقدر تربح DZ 4.2
9636 বাংলা টিভি প্রো BD Bangla TV 4.3
9637 বাংলাflix 4.2
9638 သိင်္ Astrology - Min Thein Kha BayDin 4.7
9639 ► MultiCraft ― Free Miner! 👍 4.3
9640 【Miku AR Camera】Mikuture 4.4
9641 【Ranobbe complete free】 Novelba - Free app tha... 0.0
9642 あなカレ【BL】無料ゲーム 4.7
9643 パーリーゲイツ公式通販｜EJ STYLE（イージェイスタイル） 0.0
9644 中国語 AQリスニング 0.0
9645 乐屋网: Buying a house, selling a house, renting ... 3.7
9646 乗換NAVITIME　Timetable & Route Search in Japan T... 4.4
9647 哈哈姆特不EY 0.0
9648 日本AV历史 4.1
9649 漫咖 Comics - Manga,Novel and Stories 4.1
9650 英漢字典 EC Dictionary 4.3
9651 감성학원 BL 첫사랑 4.4
9652 뽕티비 - 개인방송, 인터넷방송, BJ방송 0.0
9653 💎 I'm rich 3.8
9654 💘 WhatsLov: Smileys of love, stickers and GIF 4.6
9655 📏 Smart Ruler ↔️ cm/inch measuring for homework! 4.0
9656 🔥 Football Wallpapers 4K | Full HD Backgrounds 😍 4.7

[9657 rows x 2 columns]

In [23]:
top_rated_app = app_groups[app_groups.RATING == app_groups.RATING.max()]

In [24]:
print(app_groups['APP'].max())
print(top_rated_app['APP'].values)

🔥 Football Wallpapers 4K | Full HD Backgrounds 😍
['ROBLOX']

In [25]:
print(top_rated_app['APP'].values)

['ROBLOX']

In [26]:
top_rated_app_categories = app_info_df[app_info_df['APP'].values == top_rated_app['APP'].values]



In [27]:
print(top_rated_app_categories)

APP CATEGORY RATING ... lAST_UPDATED CURRENT_VERSION ANDROID_VERSION
117 ROBLOX GAME 4.5 ... July 31, 2018 2.347.225742 4.1 and up
471 ROBLOX GAME 4.5 ... July 31, 2018 2.347.225742 4.1 and up
2993 ROBLOX GAME 4.5 ... July 31, 2018 2.347.225742 4.1 and up
3742 ROBLOX FAMILY 4.5 ... July 31, 2018 2.347.225742 4.1 and up
4088 ROBLOX FAMILY 4.5 ... July 31, 2018 2.347.225742 4.1 and up
4190 ROBLOX GAME 4.5 ... July 31, 2018 2.347.225742 4.1 and up
4844 ROBLOX FAMILY 4.5 ... July 31, 2018 2.347.225742 4.1 and up
5124 ROBLOX FAMILY 4.5 ... July 31, 2018 2.347.225742 4.1 and up
5436 ROBLOX GAME 4.5 ... July 31, 2018 2.347.225742 4.1 and up

[9 rows x 13 columns]

In [28]:
categories_with_ratings = top_rated_app_categories.groupby('CATEGORY')['RATING'].sum()


In [29]:
print(categories_with_ratings)

CATEGORY
FAMILY 18.0
GAME 22.5
Name: RATING, dtype: float32